$\textbf{Empirical Exercise-Week 5}$  Page 308, Problem 6.29
Let's look at inequality and tax rates on income 

We will do an example of the Chow test for a simple model of income inequality on the tax rate for US Data. You can use other country data as a check on robustness.  

In [1]:
addpath(genpath('c:\LeSage7'));
Tab1 = readtable('inequality.csv');

In [2]:
Tab1(1,:)


ans =

  1x17 table

    year    aus_gdp    aus_gwth    aus_share    aus_tax    ca_gdp    ca_gwth    ca_share    ca_tax    nz_gdp    nz_gwth    nz_share    nz_tax    us_gdp    us_gwth    us_share    us_tax
    ____    _______    ________    _________    _______    ______    _______    ________    ______    ______    _______    ________    ______    ______    _______    ________    ______

     1      4910.8      3.0388       11.63       11.1       3357     -13.064      17.6       4.2      5128.1    -8.2824     15.47       10.5     5322.7    -4.1351     15.47        12  




In [3]:
year = Tab1(:,1); share = Tab1(:,16); tax = Tab1(:,17); growth = Tab1(:,15);

In [4]:
year1 = table2array(year); share1 = table2array(share); tax1 = table2array(tax); 
growth1 = table2array(growth);

First do a regression on the full sample.

In [5]:
y = log(share1);
xx =  [ones(length(y),1) tax1];
result1 = ols(y,xx);
[result1.beta result1.tstat];
result1
[result1.beta result1.tstat]




result1 = 

  struct with fields:

     meth: 'ols'
        y: [80x1 double]
     nobs: 80
     nvar: 2
     beta: [2x1 double]
     yhat: [80x1 double]
    resid: [80x1 double]
     sige: 0.0238
     bstd: [2x1 double]
     bint: [2x2 double]
    tstat: [2x1 double]
     rsqr: 0.7302
     rbar: 0.7267
       dw: 0.1916


ans =

    2.9569   72.9519
   -0.0151  -14.5279




We see that the tax rate has a negative effect on the share of the highest one percent.   calculate the sum of squared residuals.


In [6]:
SSR1 = sum((result1.resid).^2)


SSR1 =

    1.8539




The sum of squared residuals for the full sample is 1.8539. Now lets split the sample in half. 


Year 40 is 1961 since year 1 is 1921.

In [7]:
yy = log(share1(1:40,:));

tax11=tax1(1:40,:);
xxx =  [ones(length(yy),1) tax11];

result2 = ols(yy,xxx);
[result2.beta result2.tstat]
SSR2 = sum(result2.resid.^2)



ans =

    2.9020   89.5990
   -0.0126  -12.5609


SSR2 =

    0.5213




We see that the sum of squared residuals for the first half is .5213. Is this coefficient different from the later part? Let's see.

In [8]:
yyy = log(share1(40:end,:));

tax111=tax1(40:end,:);
xxxx =  [ones(length(yyy),1) tax111];

result3 = ols(yyy,xxxx);
[result3.beta result3.tstat]
SSR3 = sum(result3.resid.^2)


ans =

    3.4696   23.0798
   -0.0270   -8.0477


SSR3 =

    0.9563




We see that the residuals are different and that the coefficient of the tax rate is different.  Let's do a Chow Test

In [9]:
Fchow1 =  (SSR1- (SSR2+SSR3))/2;
Fchow2 = (SSR2+SSR3)/(80-4);
Ftest= Fchow1/Fchow2


Ftest =

    9.6777




In [10]:
Pvalue = 1-fcdf(Ftest,2,76)


Pvalue =

   1.8022e-04




We see that that is evidence of structural change in the way tax rates affect inequalty before and after 1961. Tax rates are more effective after 1961 than before.

Higher tax rates reduce the share of national income of the top 1%. No let's go back to main sample.  Run a regression on year and year-squared as well as constant term and tax rate.

In [11]:
xxx = [xx year1 year1.^2];
result2 = ols(y,xxx);
[result2.beta result2.tstat] % year 1 is for 1921


ans =

    3.1325   76.2059
   -0.0080   -5.0849
   -0.0272   -6.7142
    0.0003    7.3179




In [12]:
year_min = - .5* result2.beta(3,1)/result2.beta(4,1)


year_min =

   43.7508




Share minimizing year is 1965 since we start at 1921.

95% confidence interval for the share minizing year

In [8]:
[-.5* result2.bint(3,1)/result2.bint(4,1); -.5* result2.bint(3,2)/result2.bint(4,2)]


ans =

   77.9420
   24.1892




Confidence interval for share-minimizing years: 1945 and 1999.

In [13]:
xxxx = [xxx growth1]; % Add a growth term
result3 = ols(y,xxxx);
[result3.beta result3.tstat]


ans =

    3.1264   76.4003
   -0.0081   -5.2119
   -0.0271   -6.7387
    0.0003    7.3554
    0.0035    1.5526




Growth term is not significant

Find tax rate which would get the share of the top 1% to 12% in 2001.

In [14]:
tax_star = (log(12)-result3.beta(1,:) - result3.beta(3,:)*80 - result3.beta(4,1) * 80^2 - result3.beta(5,1) ...
* 3)/result3.beta(2,1)


tax_star =

   57.7310




Taxes would have to be 58% to get the share of the top 1% down to 12% of income in 2001.

In [11]:
tax_star_min = (log(12)-result3.bint(1,1) - result3.bint(3,1)*80 - result3.bint(4,1) * 80^2 - result3.bint(5,1) ...
* 3)/result3.bint(2,1)


tax_star_min =

  -71.5475




In [12]:
tax_star_max = (log(12)-result3.bint(1,2) - result3.bint(3,2)*80 - result3.bint(4,2) * 80^2 - result3.bint(5,2) ...
* 3)/result3.bint(2,2)


tax_star_max =

  346.9802




So a crazy confidence interval
Now lets do an easy Chow test.


help chowtest: just tpe in the full sample y,xx and the timing of the break point

In [17]:
help chowtest

 CHOWTEST Chow tests for structural change
 
  Syntax:
 
    [h,pValue,stat,cValue] = chowtest(X,y,bp)
    [h,pValue,stat,cValue] = chowtest(Tbl,bp)
    [h,pValue,stat,cValue] = chowtest(...,param,val,...)
 
  Description:
 
    Chow tests assess the stability of coefficients b in a multiple linear
    regression model of the form y = X*b + e. Data are split at specified
    break points in bp. Coefficients are estimated in initial subsamples,
    then tested for compatibility with data in complementary subsamples.
 
  Input Arguments:
 
    X - numObs-by-numPreds matrix of predictor data for a multiple linear                                                                                                                                                                                                                                                                                                                                                                                               

In [22]:
[h,p]= chowtest(xx(:,2),y,40)


h =

  logical

   1


p =

   7.3263e-05


